# Python for Scientific Computing

## Contents

  - [Scientific Libraries](#Scientific-Libraries)  
  - [The Need for Speed](#The-Need-for-Speed)  
  - [Vectorization](#Vectorization)  

## Scientific Libraries

One obvious reason we use scientif libraries is because they implement routines we want to use.

But another (more important) reason is that pure Python, while flexible and elegant, is not fast. So we need libraries that are designed to accelerate execution of Python code. 

### Python's Scientific Ecosystem

In terms of popularity, the big four in the world of scientific Python libraries are:
- NumPy: provides a basic array data type and functions for acting on these arrays.
- SciPy: builds on NumPy by adding the kinds of numerical methods that are routinely used in science.
- Matplotlib: generates figures, with a focus on plotting data stored in NumPy arrays.
- Pandas: provides types and functions for empirical work (e.g., manipulating data).

We will also cover Numba, which accelerates execution via JIT compilation.

## The Need for Speed

Higher level languages like Python are optimized for humans. This means that the programmer can leave many details to the runtime environment, such as

- specifying variable types  
- memory allocation/deallocation, etc.  

The upside is that, compared to low-level languages, Python is typically faster to write, less error prone and  easier to debug.

The downside is that Python is harder to optimize — that is, turn into fast machine code — than languages like C or Fortran.
Indeed, the standard implementation of Python (called CPython) cannot match the speed of compiled languages such as C or Fortran. 

Does that mean that we should just switch to C or Fortran for everything?
The answer is no, this is because

1. Of any given program, relatively few lines are ever going to be time-critical.
1. For those lines of code that *are* time-critical, we can achieve C-like speed using Python's scientific libraries.

### Where are the Bottlenecks?

Let’s start by trying to understand why high level languages like Python are slower than compiled code.

#### Dynamic Typing vs Static Types

Consider this Python operation

In [ ]:
a, b = 10, 10
a + b

Even for this simple operation, the Python interpreter has a fair bit of work to do.
For example, in the statement `a + b`, the interpreter has to know which
operation to invoke: If `a` and `b` are strings, then `a + b` requires string concatenation

In [ ]:
a, b = 'foo', 'bar'
a + b

If `a` and `b` are lists, then `a + b` requires list concatenation

In [ ]:
a, b = ['foo'], ['bar']
a + b

Here the operator + is overloaded — its action depends on the type of the objects on which it acts. 
As a result, Python must check the type of the objects and then call the correct operation. 
This involves substantial overheads.

Compiled languages avoid these overheads with explicit, static types. 
For example, consider the following C code, which sums the integers from 1 to 10.

```c
#include <stdio.h>

int main(void) {
    int i;
    int sum = 0;
    for (i = 1; i <= 10; i++) {
        sum = sum + i;
    }
    printf("sum = %d\n", sum);
    return 0;
}
```


The variables `i` and `sum` are explicitly declared to be integers. Hence, the meaning of addition here is completely unambiguous.

#### Data Access

Another drag on speed for high level languages is data access. 
To illustrate, let’s consider the problem of summing some data — say, a collection of integers.

##### Summing with Compiled Code
In C or Fortran, these integers would typically be stored in an array, which
is a simple data structure for storing homogeneous data. 
Such an array is stored in a single contiguous block of memory:

- In modern computers, memory addresses are allocated to each byte (one byte = 8 bits)  
- For example, a 64 bit integer is stored in 8 bytes of memory  
- An array of $ n $ such integers occupies $ 8n $ **consecutive** memory slots  

Moreover, the compiler is made aware of the data type by the programmer, in this case 64 bit integers.  
Hence, each successive data point can be accessed by shifting forward in memory
space by a known and fixed amount, in this case 8 bytes.

##### Summing in Pure Python
Python tries to replicate these ideas to some degree. 
For example, in the standard Python implementation (CPython), list elements are placed in memory locations that are in a sense contiguous. 

However, these list elements are more like pointers to data rather than actual data. 
Hence, there is still overhead involved in accessing the data values themselves. 
This is a considerable drag on speed.
In fact, it’s generally true that memory traffic is a major culprit when it comes to slow execution. 

## Vectorization

There is a clever method called **vectorization** that can be used to speed up high level languages in numerical applications.

The key idea is to send array processing operations in batch to pre-compiled and efficient native machine code. The machine code itself is typically compiled from carefully optimized C or Fortran.

This clever idea dates back to MATLAB, which uses vectorization extensively.
This can greatly accelerate many (but not all) numerical computations.

Let's see how vectorization works in Python, using NumPy.

### Operations on Arrays

First let’s run some imports

In [ ]:
import random
import numpy as np
import time

Now let’s try this non-vectorized code

In [ ]:
start = time.time() # record start time

n = 100000
sum = 0
for i in range(n):
    x = random.uniform(0, 1)
    sum += x**2
    
end = time.time() # record end time

elapsed_time = end - start # test timing
print(elapsed_time)   

The followng vectorized code achieves the same thing.

In [ ]:
start = time.time() # record start time

n = 100000
x = np.random.uniform(0, 1, n)
np.sum(x**2)

end = time.time() # record end time

elapsed_time = end - start # test timing
print(elapsed_time)   

The second code block runs much faster. 
The reason is that in the second implementation we have broken the loop down into three basic operations:

1. draw `n` uniforms  
1. square them  
1. sum them  

These are sent as batch operators to optimized machine code.
Apart from minor overheads associated with sending data back and forth, the result is C or Fortran-like speed. 

When we run batch operations on arrays like this, we say that the code is *vectorized*. Vectorized code is typically fast and efficient. It is also surprisingly flexible, in the sense that many operations can be vectorized.

### Universal Functions

Many functions provided by NumPy are so-called *universal functions* — also called [ufuncs](https://docs.scipy.org/doc/numpy/reference/ufuncs.html)

This means that they

- map scalars into scalars, as expected  
- map arrays into arrays, acting element-wise  

For example, `np.cos` is a ufunc:

In [ ]:
np.cos(1.0)

In [ ]:
np.cos(np.linspace(0, 1, 3))

By exploiting ufuncs, many operations can be vectorized.

For example, consider the problem of maximizing a function $ f $ of two
variables $ (x,y) $ over the square $ [-a, a] \times [-a, a] $.
For $ f $ and $ a $ let’s choose:

$$
f(x,y) = \frac{\cos(x^2 + y^2)}{1 + x^2 + y^2}
\quad \text{and} \quad
a = 3
$$

To maximize $f$, we’re going to use a naive grid search:

1. Evaluate $ f $ for all $ (x,y) $ in a grid on the square  
1. Return the maximum of observed values  

Here’s a non-vectorized version that uses Python loops:

In [ ]:
def f(x, y):
    return np.cos(x**2 + y**2) / (1 + x**2 + y**2)

grid = np.linspace(-3, 3, 1000)
m = -np.inf

start = time.time() # record start time

### non-vectorized version
for x in grid:
    for y in grid:
        z = f(x, y)
        if z > m:
            m = z
            
end = time.time() # record end time

elapsed_time = end - start # test timing
print(elapsed_time)  

And here’s a vectorized version:

In [ ]:
def f(x, y):
    return np.cos(x**2 + y**2) / (1 + x**2 + y**2)

grid = np.linspace(-3, 3, 1000)
x, y = np.meshgrid(grid, grid)

start = time.time() # record start time

### vectorized version
np.max(f(x, y))

end = time.time() # record end time

elapsed_time = end - start # test timing
print(elapsed_time)  

In the vectorized version, all the looping takes place in compiled code, which makes it much faster.

### Pros and Cons of Vectorization

At its best, vectorization yields fast, simple code. 
However, it’s not without disadvantages.

- One issue is that it can be highly memory intensive. 

For example, the vectorized maximization routine above is far more memory intensive than the non-vectorized version that preceded it. 
This is because vectorization tends to create many intermediate arrays before producing the final calculation.

- Another issue is that not all algorithms can be vectorized. 

In these kinds of settings, we need to go back to loops. 
Fortunately, there are alternative ways to speed up Python loops that work in almost any setting. 
One example is through Numba that can generate extremely fast and efficient code through **just in time (JIT) compilation**. 